# VQE y problemas de optimización

In [28]:
%pip install qiskit qiskit-algorithms qiskit-optimization qiskit-aer

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer

# Algoritmos Cuánticos
from qiskit_algorithms import SamplingVQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals

# Componentes de Circuito y Primitivas (Versión Qiskit 1.0+)
from qiskit.circuit.library import EfficientSU2
from qiskit.primitives import StatevectorSampler # <--- ESTA es la clave para Qiskit 1.0+

# --- 1. Definición del Problema (Datos de Entrada) ---
print("--- 1. INICIALIZANDO PROBLEMA UFLP ---")
num_bodegas = 2
num_clientes = 2

# Costos: Bodega 0 barata (10), Bodega 1 cara (20)
costos_instalacion = [10, 20] 

# Costos transporte:
# Bodega 0 -> Cliente 0 (1), Cliente 1 (5)
# Bodega 1 -> Cliente 0 (5), Cliente 1 (1)
costos_transporte = [
    [1, 5], 
    [5, 1]   
]
print(f"Instancia: {num_bodegas} bodegas, {num_clientes} clientes.")

--- 1. INICIALIZANDO PROBLEMA UFLP ---
Instancia: 4 bodegas, 4 clientes.


In [30]:
# --- 2. Modelado Matemático (QuadraticProgram) ---
qp = QuadraticProgram()

# Variables
for i in range(num_bodegas):
    qp.binary_var(name=f"x_{i}") # Bodegas
for i in range(num_bodegas):
    for j in range(num_clientes):
        qp.binary_var(name=f"y_{i}_{j}") # Enlaces

# Función Objetivo
linear_terms = {}
for i in range(num_bodegas):
    linear_terms[f"x_{i}"] = costos_instalacion[i]
for i in range(num_bodegas):
    for j in range(num_clientes):
        linear_terms[f"y_{i}_{j}"] = costos_transporte[i][j]
qp.minimize(linear=linear_terms)

# Restricciones
# 1. No asignar a bodega cerrada (y_ij <= x_i)
for i in range(num_bodegas):
    for j in range(num_clientes):
        qp.linear_constraint(linear={f"y_{i}_{j}": 1, f"x_{i}": -1}, sense="LE", rhs=0, name=f"link_{i}_{j}")

# 2. Cada cliente atendido por EXACTAMENTE una bodega (Sum(y_ij) == 1)
for j in range(num_clientes):
    linear_dict = {f"y_{i}_{j}": 1 for i in range(num_bodegas)}
    qp.linear_constraint(linear=linear_dict, sense="EQ", rhs=1, name=f"assign_{j}")

print("\nModelo clásico (primeras líneas):")
print(str(qp.prettyprint())[:300] + "...\n")

IndexError: list index out of range

In [ ]:
# --- 3. Conversión a Hamiltoniano ---
conv = QuadraticProgramToQubo()
qubo = conv.convert(qp)
op, offset = qubo.to_ising()

print(f"Número de Qubits necesarios: {op.num_qubits}")

Número de Qubits necesarios: 6


In [ ]:
# --- 4. Configuración y Ejecución del VQE ---
print("\n--- EJECUTANDO VQE (Qiskit 1.0 Sampling) ---")

# A. Semilla (Reproducibilidad)
algorithm_globals.random_seed = 12345

# B. Ansatz
ansatz = EfficientSU2(num_qubits=op.num_qubits, reps=2, entanglement='linear')

# C. Optimizador
optimizer = COBYLA(maxiter=300) # Aumentamos un poco las iteraciones para asegurar convergencia

# D. Instanciar SamplingVQE con StatevectorSampler
# StatevectorSampler es el simulador ideal matemático incluido en Qiskit
sampler = StatevectorSampler(seed=algorithm_globals.random_seed)
vqe = SamplingVQE(sampler=sampler, ansatz=ansatz, optimizer=optimizer)

# E. Resolver
vqe_solver = MinimumEigenOptimizer(vqe)
result = vqe_solver.solve(qp)

# --- 5. Análisis de Resultados ---
print("\n--- RESULTADOS FINALES ---")
print(f"Estado óptimo (raw): {result.x}")
print(f"Costo Mínimo Encontrado: {result.fval}")

# Interpretación para la presentación
print("\n--- INTERPRETACIÓN ---")
variables_activas = [var.name for var, val in zip(qp.variables, result.x) if val == 1]
print(f"Decisión tomada: {variables_activas}")

# Validación rápida
bodegas_abiertas = [v for v in variables_activas if 'x_' in v]
enlaces_activos = [v for v in variables_activas if 'y_' in v]
print(f"Bodegas abiertas: {len(bodegas_abiertas)}")
print(f"Clientes asignados: {len(enlaces_activos)}")

if len(enlaces_activos) == num_clientes and len(bodegas_abiertas) > 0:
    print("✅ SOLUCIÓN VÁLIDA: Todos los clientes tienen asignación.")
else:
    print("⚠️ SOLUCIÓN INVÁLIDA: Revisa las penalizaciones o aumenta las iteraciones.")


--- EJECUTANDO VQE (Qiskit 1.0 Sampling) ---


C:\Users\yamca\AppData\Local\Temp\ipykernel_16900\1600317301.py:8: DeprecationWarning: The class ``qiskit.circuit.library.n_local.efficient_su2.EfficientSU2`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.efficient_su2 instead.
  ansatz = EfficientSU2(num_qubits=op.num_qubits, reps=2, entanglement='linear')



--- RESULTADOS FINALES ---
Estado óptimo (raw): [0. 1. 0. 0. 1. 1.]
Costo Mínimo Encontrado: 26.0

--- INTERPRETACIÓN ---
Decisión tomada: ['x_1', 'y_1_0', 'y_1_1']
Bodegas abiertas: 1
Clientes asignados: 2
✅ SOLUCIÓN VÁLIDA: Todos los clientes tienen asignación.
